# Testing software

In [1]:
import numpy as np
from matplotlib import pyplot as plt

## Sources

Decided to go with the astromodels package as python3 branch is working well.

To install: 
```
pip install git+https://github.com/giacomov/astromodels.git@py2to3
```

In [75]:
from source.source import *
from astromodels.functions.functions import Powerlaw
from astromodels import Model

In [76]:
# Single source
source = PointSource('point', ra=123, dec=-30, redshift=0.1, spectral_shape=Powerlaw())
source.position.get_ra()
source.redshift

0.1

In [77]:
# Loading from file
source_file = '../dev/statistical_model/data/test_SFR_pop.h5'
source_list = TestSourceList(source_file, spectral_shape=Powerlaw())

# Select sources below zth = 1
zth = 1.0
source_list.select_below_redshift(zth)

# Add a diffuse background at z = 1
diffuse_bg = DiffuseSource('diffuse_bg', zth, spectral_shape=Powerlaw())
source_list.add(diffuse_bg)

In [78]:
source_list.sources[0].position.dec.value

-19.438230205811653

In [79]:
source_list.sources[-1].redshift
source_list.N

38

In [7]:
# Could also use an astromodels model
source_model = Model(*source_list.sources)

## Precomputation

In [8]:
from Precomputation import ExposureIntegral
from detector_model import NorthernTracksEffectiveArea

In [9]:
time = 8 # years
Emin = 1.0e5 # GeV

exposure_integral = ExposureIntegral(source_list, NorthernTracksEffectiveArea, 
                                     time, Emin)
exposure_integral()

## Generate Stan code for simulation


In [12]:
from detector_model import NorthernTracksDetectorModel
from backend.stan_generator import (
    StanGenerator, GeneratedQuantitiesContext, DataContext, TransformedDataContext,
    FunctionsContext, Include, ForLoopContext, StanFileGenerator)
from backend.variable_definitions import ForwardVariableDef, ForwardArrayDef
from backend.expression import StringExpression
import logging
logging.basicConfig(level=logging.WARN)

In [70]:
with StanGenerator() as cg:

    with FunctionsContext() as fc:
        _ = Include("utils.stan")
        _ = Include("vMF.stan")
        _ = Include("interpolation.stan")
        _ = Include("sim_functions.stan")
        _ = Include("energy_spectrum.stan")
        ntd = NorthernTracksDetectorModel()
            
    with DataContext() as dc:
        
        # Sources
        Ns = ForwardVariableDef("Ns", "int")
        Ns_str = ["[", Ns, "]"]
        Ns_1p_str = ["[", Ns, "+1]"]
        
        varpi = ForwardArrayDef("varpi", "unit_vector[3]", Ns_str)
        D = ForwardVariableDef("D", "vector[Ns]")
        redshift = ForwardVariableDef("z", "vector[Ns+1]")
        
        # Energies
        alpha = ForwardVariableDef("alpha", "real")
        Emin = ForwardVariableDef("Emin", "real")
        
        # Flux
        Q = ForwardVariableDef("Q", "real")
        F0 = ForwardVariableDef("F0", "real")
        
        # Precomputed quantities
        Ngrid = ForwardVariableDef("Ngrid", "int")
        Ngrid_str = ["[", Ngrid, "]"]
        alpha_grid = ForwardVariableDef("alpha_grid", "vector[Ngrid]")
        integral_grid = ForwardArrayDef("integral_grid", 
                                        "vector[Ngrid]", Ns_1p_str)
        aeff_max = ForwardVariableDef("aeff_max", "real")
        
        # Inputs
        T = ForwardVariableDef("T", "real")
        
    with TransformedDataContext() as tdc:
        F = ForwardVariableDef("F", "vector[Ns+1]")
        FT = ForwardVariableDef("FT", "real")
        Fs = ForwardVariableDef("Fs", "real")
        f = ForwardVariableDef("f", "real")
        w_exposure = ForwardVariableDef("w_exposure", "simplex[Ns+1]")
        Nex = ForwardVariableDef("Nex", "real")
        N = ForwardVariableDef("N", "int")
        eps = ForwardVariableDef("eps", "vector[Ns+1]")
        
        Mpc_to_m = ForwardVariableDef("Mpc_to_m", "real")
        Mpc_to_m << StringExpression("3.086e22")
        
        with ForLoopContext(1, Ns, "k") as k:
            F[k] << StringExpression([Q, "/ 4 * pi() * pow(", 
                                         D[k], " * ", Mpc_to_m, ", 2)"])
            StringExpression([Fs," += ", F[k]])
        
        StringExpression("F[Ns+1]") << F0
        
        FT << Fs + FT
        f << StringExpression([Fs, "/", FT])
        
        eps << StringExpression(["get_exposure_factor(", T, ", ", Emin, ", ", alpha, 
                                 ", ", alpha_grid, ", ", integral_grid, ", ", Ns, ")"])
        Nex << StringExpression(["get_Nex_sim(", F, ", ", eps, ")"])
        w_exposure << StringExpression(["get_exposure_weights(", F, ", ", 
                                              eps, ")"])
        N << StringExpression(["poisson_rng(", Nex, ")"])
        
    with GeneratedQuantitiesContext() as gqc:
        
        N_str = ["[", N, "]"]
        lam = ForwardArrayDef("Lambda", "int", N_str)
        omega = ForwardVariableDef("omega", "unit_vector[3]")
        
        Esrc = ForwardVariableDef("Esrc", "vector[N]")
        E = ForwardVariableDef("E", "vector[N]")
        Edet = ForwardVariableDef("Edet", "vector[N]")
        
        zenith = ForwardArrayDef("zenith", "real", N_str)
        Pdet = ForwardArrayDef("Pdet", "real", N_str)
        accept = ForwardVariableDef("accept", "real")
        prob = ForwardVariableDef("prob", "simplex[2]")
        
        event = ForwardArrayDef("event", "unit_vector[3]", N_str)
        Nex_sim = ForwardVariableDef("Nex_sim", "real")
        
        Nex_sim << Nex
        StringExpression(["print(", Ngrid, ")"])
        StringExpression(["print(", N, ")"])

        with ForLoopContext(1, N, "i") as i:
            
            accept << 0
            
            StringExpression(["while(", accept, " != 1) {\n", 
                              "if (", lam[i], " < Ns+1) {\n",
                              omega, " = ", varpi[lam[i]], ";\n",
                              "}\n", "else if (", lam[i], " == Ns+1) {\n", 
                              omega, "= sphere_rng(1);\n", "}"])

            zenith[i] << StringExpression(["omega_to_zenith(", omega, ")"])
            
            # Sample energy
            Esrc[i] << StringExpression(["spectrum_rng(", alpha, ", ", 
                                         Emin, " * (1+", redshift[lam[i]], ") )"])
            E[i] << StringExpression([Esrc[i], "/ (1+", redshift[lam[i]], ")"])
            
            # Test against Aeff
            Pdet[i] << StringExpression([ntd.effective_area(E[i], omega), 
                                         " / ", aeff_max])
            prob[1] << Pdet[i]
            prob[2] << 1 - Pdet[i]
            
            accept << StringExpression(["categorical_rng(", prob, ")"])
            
            StringExpression("}")
            
            # Detection effects (simple test case for now)
            event[i] << StringExpression(["vMF_rng(", omega, ", 100)"])
            Edet[i] << StringExpression(["lognormal_rng(log(", E[i], "), 0.3)"])
        
        
    sim = cg.generate()

In [74]:
#print(sim)

In [73]:
# Compile
this_dir = os.path.abspath('')
include_paths = [os.path.join(this_dir, 
                              "../dev/statistical_model/4_tracks_and_cascades/stan/")]
sm = pystan.StanModel(
    model_code=sim,
    include_paths=include_paths,
    verbose=False)

In [81]:
# Inputs 
inputs = {}
inputs['Ns'] = source_list.N